# Use GPU

In [1]:
#%run /Desktop/Share/CUDA_DEVICE_setup.py -n 1

using GPU ID:0,



# Import packages

In [2]:
import matplotlib
import matplotlib.pyplot as plt

In [3]:
import os
import deepcalo as dpcal
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# ==============================================================================
# Argument, for further available arguments please refer to the readme file
# ==============================================================================

exp_dir = 'exp_electrons/'
data_path = '/Desktop/CodeFolder/crchen/MC_abseta_1.6_2.5_et_0.0_5000000.0_processes_energy.h5'
rm_bad_reco = True
zee_only = True

gpu_ids = "0"
apply_mask = rm_bad_reco or zee_only 

2022-12-02 00:40:47.633653: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# Import parameters config

In [4]:
from importlib import import_module
param_conf = import_module(exp_dir.split('/')[-2] + '.param_conf')
params = param_conf.get_params()
params = dpcal.utils.merge_dicts(dpcal.utils.get_default_params(), params, in_depth=True)
if params['auto_lr']:
    params = dpcal.utils.set_auto_lr(params)
dirs = dpcal.utils.create_directories(exp_dir, params['epochs'])

Learning rate range for learning rate schedule CLR set to (4.61880e-04, 1.38564e-03) by auto_lr.


In [5]:
if 'quantizer_config' in params:
    print("quantized config found:")
    print("kernel_quantizer: \t", params['quantizer_config']['kernel_quantizer'])
    print("bias_quantizer: \t", params['quantizer_config']['bias_quantizer'])
    print("activation_bits: \t", params['quantizer_config']['activation_bits'])
    print("q_activation: \t\t", params['quantizer_config']['q_activation'])
    print("final_q_activation: \t", params['quantizer_config']['final_q_activation'])
    from qkeras import QActivation
    max_quantized_value = QActivation(params['quantizer_config']['activation_bits']).quantizer.max()
    min_quantized_value = QActivation(params['quantizer_config']['activation_bits']).quantizer.min()
    print("max_quantized_value: \t", max_quantized_value)
    print("min_quantized_value: \t", min_quantized_value)

quantized config found:
kernel_quantizer: 	 quantized_bits(16,7,alpha=1)
bias_quantizer: 	 quantized_bits(16,7,alpha=1)
activation_bits: 	 quantized_bits(16,7,alpha=1)
q_activation: 		 quantized_relu(bits=16, integer=8)
final_q_activation: 	 quantized_relu(bits=32, integer=16)
max_quantized_value: 	 128.0
min_quantized_value: 	 -128.0


2022-12-02 00:40:49.814813: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-12-02 00:40:49.815701: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-12-02 00:40:49.816504: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuil

# Standardize scalars and mask tracks

In [6]:
_n_train = 8e5
_n_val = -1
_n_test = -1

In [ ]:

n_points = {'train':None, 'val':None, 'test':None}
for set_name, n in zip(n_points, [_n_train, _n_val, _n_test]):
    if not n < 0:
        n_points[set_name] = n
print(n_points)
if params['data_generator']['use']:
    params['data_generator']['n_points'] = n_points
    n_points = {set_name:1 for set_name in n_points}
    if data_path is not None:
        params['data_generator']['path'] = data_path
    data = dpcal.utils.load_atlas_data(n_points=n_points,
                                       **params['data_generator']['load_kwargs'],
                                       verbose=False)
else:
    # Import data parameters (what should or should not be loaded) as a module
    data_conf = import_module(exp_dir.split('/')[-2] + '.data_conf')
    data_params = data_conf.get_params()

    # Load the data
    data = dpcal.utils.load_atlas_data(path=data_path, n_points=n_points, use_hls4ml = params['use_hls4ml'], **data_params)
scalars_to_quantile = ['p_pt_track', 'p_R12', 'p_deltaPhiRescaled2', 'p_deltaEta2',
                        'p_f0Cluster', 'p_eAccCluster', 'p_photonConversionRadius',]

# Standardize scalars and tracks
for dataset_name in ['scalars']:

    if dataset_name=='scalars':
        data_param_name = 'scalar_names'
    elif dataset_name=='tracks':
        data_param_name = 'track_names'
    if data_params[data_param_name] is None:
        continue

    # Make directory for saving scalers
    scaler_dir = dirs['log'] + 'scalers/'
    if not os.path.exists(scaler_dir):
        os.makedirs(scaler_dir)

    for name in data_params[data_param_name]:
        # Get index of variable
        var_ind = data_params[data_param_name].index(name)

        # Get which scaler to use
        scaler_name = 'Quantile' if name in scalars_to_quantile else 'Robust'

        # Standardize in-place
        dpcal.utils.standardize(data,
                                dataset_name,
                                variable_index=var_ind,
                                scaler_name=scaler_name,
                                save_path=os.path.join(scaler_dir, f'scaler_{scaler_name}_{name}.jbl'))

        if True:
            print(f'Standardizing {name} with {scaler_name}Scaler')
            for set_name in data:
                print(f'Min and max of {name} in {set_name} set after standardization: '
                      f'{data[set_name][dataset_name][:,var_ind].min(), data[set_name][dataset_name][:,var_ind].max()}')
                
if not params['data_generator']['use']:

    if apply_mask:
        # Make masks
        mask = dpcal.utils.make_mask(data_path, n_points, rm_bad_reco=rm_bad_reco,
                                     zee_only=zee_only, lh_cut_name=None,
                                     rm_conv=None, rm_unconv=None)

        # Apply mask
        data = dpcal.utils.apply_mask(data, mask, skip_name=None)
        
        # set apply_mask to False in parameters since it is aleardy masked
        params['track_net']['apply_mask'] = False

{'train': 800000.0, 'val': None, 'test': None}
Loading data.
Loading only the 800000 first data points of the train set.
Since using hls4ml later, Targets will be scaled down by 1


# Data normalization

In [ ]:
# images:

for set_name in n_points:
    imgs = data[set_name]['images']['em_endcap']
    imgs_thr = np.percentile(np.max(imgs, axis=(1,2,3)) ,95)
    imgs = np.clip(imgs, None, imgs_thr)
    data[set_name]['images']['em_endcap'] = MinMaxScaler().fit_transform(imgs.reshape(imgs.shape[0], -1)).reshape(imgs.shape)
    

In [ ]:
# scalars:

for set_name in n_points:
    data_pre = data[set_name]['scalars']
    data_pre_thr = np.percentile(np.max(data_pre, axis=1) ,95)
    data_pre = np.clip(data_pre, None, data_pre_thr)
    data[set_name]['scalars'] = MinMaxScaler().fit_transform(data_pre.reshape(data_pre.shape[0], -1)).reshape(data_pre.shape)

In [ ]:
# tracks:

for set_name in n_points:
    data_pre = data[set_name]['tracks']
    data_pre_thr = np.percentile(np.max(data_pre, axis=(1,2)) ,95)
    data_pre = np.clip(data_pre, None, data_pre_thr)
    data[set_name]['tracks'] = MinMaxScaler().fit_transform(data_pre.reshape(data_pre.shape[0], -1)).reshape(data_pre.shape)

# Plot max value in each batch

In [ ]:

tr_imgs = data["train"]['images']['em_endcap']
tr_scalars = data["train"]['scalars']
tr_tracks = data["train"]['tracks']
tr_targets = data["train"]['targets']


In [ ]:
print("Min and max of images: ", np.min(tr_imgs), np.max(tr_imgs))
print("Min and max of scalars: ", np.min(tr_scalars), np.max(tr_scalars))
print("Min and max of tracks: ", np.min(tr_tracks), np.max(tr_tracks))
print("Min and max of targets: ", np.min(tr_targets), np.max(tr_targets))


In [ ]:
#tr_tracks_bool = np.max(tr_tracks, axis=(1,2)) > np.percentile(np.max(tr_tracks, axis=(1,2)) ,90)

In [ ]:
#tr_tracks = np.delete(tr_tracks, tr_tracks_bool,axis = 0 )

In [ ]:
#q95,q5 = np.percentile(tr_imgs,[95,5])
#intr_qr = q95-q5
#print(intr_qr) 

In [ ]:
fontsize = 30
plt.figure(figsize = (20,15))
plt.plot(np.max(tr_imgs, axis=(1,2,3)) )
plt.title('max value in images/batch', fontsize = fontsize)
plt.ylabel('value', fontsize = fontsize)
plt.xlabel('batch', fontsize = fontsize)
plt.savefig("tr_images.png")
plt.show()

In [ ]:
fontsize = 30
plt.figure(figsize = (20,15))
plt.plot(np.max(tr_tracks, axis=(1,2)) )
plt.title('max value in tracks/batch', fontsize = fontsize)
plt.ylabel('value', fontsize = fontsize)
plt.xlabel('batch', fontsize = fontsize)
plt.savefig("tr_tracks.png")
plt.show()

In [ ]:
fontsize = 30
plt.figure(figsize = (20,15))
plt.plot(np.max(tr_scalars, axis=1) )
plt.title('max value in scalars/batch', fontsize = fontsize)
plt.ylabel('value', fontsize = fontsize)
plt.xlabel('batch', fontsize = fontsize)
plt.savefig("tr_scalars.png")
plt.show()

In [ ]:
fontsize = 30
plt.figure(figsize = (20,15))
plt.plot(tr_targets)
plt.title('max value in targets/batch', fontsize = fontsize)
plt.ylabel('value', fontsize = fontsize)
plt.xlabel('batch', fontsize = fontsize)
plt.savefig("tr_targets.png")
plt.show()

In [ ]:
mc = dpcal.ModelContainer(data=data,
                          params=params,
                          dirs=dirs,
                          save_figs=False,
                          verbose=True)

# Qmodel_Summary

In [ ]:
from qkeras.autoqkeras.utils import print_qmodel_summary
print_qmodel_summary(mc.model)

# Plot model

In [ ]:
from tensorflow.keras.utils import plot_model

dot_img_file = 'dcalo_full_qkeras_qdensebatchnorm.png'
plot_model(mc.model, to_file=dot_img_file, show_shapes=True)

In [ ]:
history = mc.train()

# Evaluate (predicting and evaluating on test or validation set)
if not hasattr(mc,'evaluation_scores'):
    mc.evaluate()

# Print results
print('Evaluation scores:')
print(mc.evaluation_scores)



qmodel = mc.model
print('final model saved')
qmodel.save('./model_weight/keras_qmodel_endcap_{}.h5'.format(params['quantizer_config']['activation_bits']))



In [ ]:
set_name = 'test'

In [ ]:
#input images
x_test_em_barrel = data[set_name]['images']['em_barrel']
#np.save('x_test_em_barrel.npy', x_test_em_barrel)
print('x_test_em_barrel shape: ', x_test_em_barrel.shape)
#print('x_test_em_barrel saved')

#input scalar
x_test_scalars = data[set_name]['scalars']
#np.save('x_test_scalars.npy', x_test_scalars)
print('x_test_scalars shape: ', x_test_scalars.shape)
#print('x_test_scalars saved')

#input track
x_test_tracks = data[set_name]['tracks']
#np.save('x_test_tracks.npy', x_test_tracks)
print('x_test_tracksl shape: ', x_test_tracks.shape)
#print('x_test_tracks saved')

#target
y_test_targets = data[set_name]['targets']
#np.save('y_test_targets.npy', y_test_targets)
print('y_test_targets shape: ', y_test_targets.shape)
#print('y_test_targets saved')


In [ ]:
x_test = [x_test_em_barrel, x_test_scalars, x_test_tracks]

In [ ]:
mc.model.evaluate(x_test, y_test_targets)

In [ ]:
# plot training loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.ylim(0, 80)
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('model_loss_300ep_keras_qmodel_endcap_17_9.png')
#plt.show()

In [ ]:
#quantized_model_debug(mc.model, x_test, plot=True, plt_instance=plt)